In [1]:
import numpy as np
import itertools
import matplotlib.pyplot as plt

a good start would be to look at some randomly simulated games, where each player just picks a point on a board

I want to make a function that takes a matrix as imput, and then applys the algorithm i came up with to return an optimal move, inserting 0 somewhere
we can then play against it, or run a random 1s player against it to see if it makes any mistakes

In [99]:

#first we have to do some bookkeeping on the matrix, eg checking that all elements are suitable, and that there are not too many 1s or 0s to ensure its a valid playing position

ALLOWED = {0, 1, "*", "0", "1"}

def _normalise_to_object_array(mat):
    A = np.array(mat, dtype=object, copy=True)

    #ensure that matrix is structured properly

    if A.ndim != 2 or A.shape[0] != A.shape[1]:
        raise ValueError("Matrix must be square (n×n).")
    if A.shape[0] < 4:
        raise ValueError("Strategy requires n > 3 (i.e. n >= 4).")

    # ensure that the contents of the matrix is correct and remove any accidental commas
    
    flat = A.ravel()
    for k, x in enumerate(flat):
        if x not in ALLOWED:
            raise ValueError(f"Invalid entry {x!r} at flat index {k}. Allowed: 0, 1, '*', '0', '1'.")
        if x == "0":
            flat[k] = 0
        elif x == "1":
            flat[k] = 1
        elif x == "*":
            flat[k] = "*"
    return A

In [95]:
def p0_move(mat):

    key_rows=(0, 1, 2, 3)

    A = _normalise_to_object_array(mat)
    n = A.shape[0]
    r1, r2, r3, r4 = key_rows
    pairs = [(r1, r2), (r3, r4)]
    key_set = set(key_rows)

    #define some helper functions to be used when making moves

    # Find first (1,*) or (*,1) in either key pair and put 0 in the * partner.
    def mirror_move(board: np.ndarray):
        for a, b in pairs:
            cols = np.where((board[a, :] == 1) & (board[b, :] == "*"))[0]
            if cols.size:
                j = int(cols[0])
                B = board.copy()
                B[b, j] = 0
                return B

            cols = np.where((board[b, :] == 1) & (board[a, :] == "*"))[0]
            if cols.size:
                j = int(cols[0])
                B = board.copy()
                B[a, j] = 0
                return B
        return None
    #if possible, create (0,*) in a completely blank (*,*) column of a key pair. (we now know this doesnt work)
    def seed_move(board: np.ndarray):
        for a, b in pairs:
            cols = np.where((board[a, :] == "*") & (board[b, :] == "*"))[0]
            if cols.size:
                j = int(cols[0])
                B = board.copy()
                B[a, j] = 0
                return B
        return None

    def play_outside(board: np.ndarray):
        outside_rows = [i for i in range(n) if i not in key_set]
        if not outside_rows:
            return None
        sub = board[outside_rows, :]
        where_star = np.argwhere(sub == "*")
        if where_star.size:
            rr, cc = where_star[0]
            i = outside_rows[int(rr)]
            j = int(cc)
            B = board.copy()
            B[i, j] = 0
            return B
        return None

    def play_anywhere(board: np.ndarray):
        where_star = np.argwhere(board == "*")
        if where_star.size == 0:
            return None
        i, j = where_star[0]
        B = board.copy()
        B[int(i), int(j)] = 0
        return B

    #this is now the ordering for making moves
    out = mirror_move(A)
    if out is not None:
        return out

    out = seed_move(A)
    if out is not None:
        return out

    out = play_outside(A)
    if out is not None:
        return out

    out = play_anywhere(A)
    if out is not None:
        return out



In [96]:
#this is its opponent, which will make random moves. theoretically it should always loose, but for enough repetes should simulate someone new to the game playing with no real strategy.
#if p0 algo has any pitfalls this should find it with enough iterations

def randP1(mat, rng=None):
    if rng is None:
        rng = np.random.default_rng() #had issues with setitng specific random seeds, this fixed it

    A = np.array(mat, dtype=object, copy=True)

    stars = np.argwhere(A == "*") #find all *s eg list of possible moves
    if stars.size == 0:
        raise ValueError("No available '*' cells.")

    i, j = stars[rng.integers(0, stars.shape[0])] #pick a * at random and converet it to a 1
    B = A.copy()
    B[int(i), int(j)] = 1
    return B

we can now try simulating full games with our stragety for p0

In [97]:
def p1start(n):
    #produce an 'empty playing table' eg an n*n mx full of *'s
    M = np.full((n, n), "*", dtype=object)

    #play n*n moves to populate entire matrix from empty

    for i in range(n*n):
        if i % 2 == 0:
            M = randP1(M)
        if i % 2 == 1:
            M = p0_move(M)

    M = np.vectorize(lambda x: 0 if x == "0" else 1 if x == "1" else x)(M).astype(float)
        
    return M, np.linalg.det(M)

def p0start(n):
    M = np.full((n, n), "*", dtype=object)

    #play n*n moves to populate entire matrix from empty

    for i in range(n*n):
        if i % 2 == 1:
            M = randP1(M)
        if i % 2 == 0:
            M = p0_move(M)

    M = np.vectorize(lambda x: 0 if x == "0" else 1 if x == "1" else x)(M).astype(float)

    return M, np.linalg.det(M)



print(p1start(4))
print(p0start(4))


NameError: name '_normalise_to_object_array' is not defined

Now we know that a single iteratoin of the game works, lets try simulating it for a few thousand, to see if we find any slip ups from our algorithm

In [100]:
def gamesim(n,k): #simulate a game of size n*n 2 k times, k times per each player starting
    v = 0
    for i in range(k):
        a,b = p1start(n)
        if b != 0:
            print(a,b, 'p1start') #diagnostic code
            v += 1
    for i in range(k):
        a,b = p0start(n)
        if b != 0:
            print(a,b, 'p0start')
            v += 1
    return v
    
print(gamesim(4,100))
print(gamesim(5,100))
print(gamesim(6,100)) #ive reduced the values of k as i know this doesnt work, to allow notebook to run faster

0
[[1. 0. 0. 0. 0.]
 [0. 1. 0. 1. 1.]
 [0. 0. 0. 1. 0.]
 [1. 1. 1. 0. 1.]
 [1. 1. 1. 1. 0.]] 1.0 p1start
[[0. 1. 0. 0. 0.]
 [0. 0. 1. 1. 1.]
 [0. 0. 1. 0. 0.]
 [1. 1. 0. 1. 1.]
 [1. 1. 1. 0. 1.]] 1.0 p1start
[[0. 0. 0. 1. 0.]
 [1. 1. 0. 0. 1.]
 [0. 1. 0. 1. 0.]
 [1. 0. 1. 0. 1.]
 [0. 1. 1. 1. 1.]] -1.0 p1start
[[1. 0. 0. 0. 0.]
 [0. 1. 1. 1. 1.]
 [0. 0. 0. 1. 0.]
 [1. 0. 1. 0. 1.]
 [1. 1. 0. 1. 1.]] -1.0 p1start
[[0. 0. 1. 0. 0.]
 [1. 1. 0. 0. 1.]
 [0. 1. 1. 1. 1.]
 [1. 0. 0. 0. 0.]
 [1. 0. 1. 1. 1.]] 1.0 p1start
[[0. 0. 0. 1. 1.]
 [1. 0. 1. 0. 0.]
 [1. 0. 0. 1. 0.]
 [0. 1. 1. 0. 1.]
 [1. 1. 1. 0. 1.]] -1.0 p1start
[[0. 0. 0. 1. 0.]
 [1. 1. 0. 0. 1.]
 [0. 0. 1. 0. 1.]
 [1. 1. 0. 1. 0.]
 [1. 0. 1. 1. 1.]] -1.0 p1start
[[0. 1. 0. 0. 0.]
 [0. 0. 1. 1. 1.]
 [0. 0. 1. 0. 0.]
 [1. 1. 0. 0. 1.]
 [1. 1. 0. 1. 1.]] -1.0 p0start
[[0. 0. 1. 0. 0.]
 [0. 1. 0. 1. 0.]
 [1. 0. 1. 1. 1.]
 [0. 1. 0. 0. 0.]
 [0. 1. 1. 1. 1.]] 1.0 p0start
[[0. 1. 0. 0. 1.]
 [1. 0. 1. 1. 0.]
 [1. 1. 0. 1. 0.]
 [0. 0. 1. 0

it seems that our method only works for n = 4, but in theory it should work for all n, there may be an issue with my code

looking into it i see that we seem to place a 0 inside the key rows earlier then we have to, this can be taken advantage of by p1.
this makes sense as for the case n=4 all rows are key rows, thus this issue would never occur


we also run into some new floating point issues with finding the determinant of larger values of n. this can be fixed by finding the rank instead, as that will indicate the exact same thing, if less that n then det = 0
we have fixed this in the new function

In [101]:
def p0_move(mat, key_rows=(0,1,2,3)):

    A = _normalise_to_object_array(mat) #ensure matrix is suitable
    n = A.shape[0]
    r1, r2, r3, r4 = key_rows
    pairs = [(r1, r2), (r3, r4)]
    key_set = set(key_rows)

    #first lets define some helper functions for each of the moves we may want to make

    # Outside rows and whether outside is full
    outside_rows = [i for i in range(n) if i not in key_set]
    outside_full = True
    if outside_rows:
        outside_full = (np.count_nonzero(A[outside_rows, :] == "*") == 0)

    # Count columns in key pairs that look like (0,*) or (*,0)
    def open_pairs_count(board: np.ndarray):
        cnt = 0
        for a, b in pairs:
            cnt += int(np.count_nonzero((board[a, :] == 0) & (board[b, :] == "*")))
            cnt += int(np.count_nonzero((board[a, :] == "*") & (board[b, :] == 0)))
        return cnt
    
    # If a key pair has (1,*) or (*,1), insert a 0
    def mirror_move(board: np.ndarray):
        for a, b in pairs:
            cols = np.where((board[a, :] == 1) & (board[b, :] == "*"))[0]
            if cols.size:
                j = int(cols[0])
                B = board.copy()
                B[b, j] = 0
                return B

            cols = np.where((board[b, :] == 1) & (board[a, :] == "*"))[0]
            if cols.size:
                j = int(cols[0])
                B = board.copy()
                B[a, j] = 0
                return B
        return None

    #this is to let us play outside the key rows, eg if there isnt an ideal move to be made there.
    def play_outside(board: np.ndarray):
        if not outside_rows:
            return None
        sub = board[outside_rows, :]
        where_star = np.argwhere(sub == "*")
        if where_star.size:
            rr, cc = where_star[0]
            i = outside_rows[int(rr)]
            j = int(cc)
            B = board.copy()
            B[i, j] = 0
            return B
        return None
    
    # Choose a blank complementary pair (*,*) and place 0 in one cell to make (0,*).
    def create_open_pair(board: np.ndarray):
        for a, b in pairs:
            cols = np.where((board[a, :] == "*") & (board[b, :] == "*"))[0]
            if cols.size:
                j = int(cols[0])
                B = board.copy()
                B[a, j] = 0
                return B
        return None

    #now we make our actual moves!

    # Always do forced mirroring first - best case move
    out = mirror_move(A)
    if out is not None:
        return out

    #If outside is not full, and there is nowhere to mirror, then place in outside
    if not outside_full:
        out = play_outside(A)
        if out is not None:
            return out

    #in case something goes wrong, just insert a 0 wherever you can, this should never happen ideally except for a final move.
    op = open_pairs_count(A)
    if op == 0:
        out = create_open_pair(A)
        if out is None:
            # No (*,*) pairs left. Only possibility is matrix full, but we checked it isn't.
            i, j = np.argwhere(A == "*")[0]
            B = A.copy()
            B[int(i), int(j)] = 0
            return B
        return out


In [102]:
#lets try again with out new algorithm for p0

def gamesim(n,k): #simulate a game of size n*n 2 k times, k times per each player starting
    v = 0
    for i in range(k):
        a,b = p1start(n)
        if np.linalg.matrix_rank(a) == n:
            v += 1
    for i in range(k):
        a,b = p0start(n)
        if np.linalg.matrix_rank(a) == n:
            v += 1
    return v
    
print(gamesim(4,10000))
print(gamesim(5,10000))
print(gamesim(6,10000))

0
0
0


voila, our new algorithm works for n op to 6, lets try some larger values

In [21]:
for i in range(9):
        print(f'n = {i+7}, lost games: {gamesim(i+7,1000)} of 2000.')

n = 7, lost games: 0 of 2000.
n = 8, lost games: 0 of 2000.
n = 9, lost games: 0 of 2000.
n = 10, lost games: 0 of 2000.
n = 11, lost games: 0 of 2000.
n = 12, lost games: 0 of 2000.
n = 13, lost games: 0 of 2000.
n = 14, lost games: 0 of 2000.
n = 15, lost games: 0 of 2000.


Excellent. our agorithm works, both for when p0 starts and for when p1 starts.
